In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
import warnings
warnings.simplefilter('ignore')

In [3]:
#sub_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
filename = '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/2class/2_df_sub17.csv'
df = pd.read_csv(filename)
df = df.dropna(axis=0)
df

,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label
0,-0.014648,-0.001968,5.543137,32.567444,2.757263,0.0
1,-0.001236,-0.005539,5.542755,32.567444,2.742004,0.0
2,0.007278,0.017761,5.556107,32.561493,2.603149,0.0
3,0.009201,0.032181,5.558014,32.559998,2.749634,0.0
4,0.007141,-0.011032,5.564117,32.558502,2.760315,0.0
...,...,...,...,...,...,...
1593195,0.175003,0.033646,7.341385,34.592560,-2.023315,1.0
1593196,0.188828,-0.001099,7.352066,34.603302,-2.000427,1.0
1593197,0.201965,-0.007874,7.345200,34.584900,-1.994324,1.0
1593198,0.212631,-0.011856,7.351303,34.543488,-2.003479,1.0


In [4]:
X = df.drop('w_label', axis = 1)
y = df.w_label

In [5]:
X

,c_ecg,c_emg,c_eda,c_temp,c_resp
0,-0.014648,-0.001968,5.543137,32.567444,2.757263
1,-0.001236,-0.005539,5.542755,32.567444,2.742004
2,0.007278,0.017761,5.556107,32.561493,2.603149
3,0.009201,0.032181,5.558014,32.559998,2.749634
4,0.007141,-0.011032,5.564117,32.558502,2.760315
...,...,...,...,...,...
1593195,0.175003,0.033646,7.341385,34.592560,-2.023315
1593196,0.188828,-0.001099,7.352066,34.603302,-2.000427
1593197,0.201965,-0.007874,7.345200,34.584900,-1.994324
1593198,0.212631,-0.011856,7.351303,34.543488,-2.003479


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [8]:
check_for_nan = df.isnull().values.any()
print (check_for_nan)

False


In [9]:
import sklearn as sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [11]:
pd.Series(y).value_counts()

0.0    1087100
1.0     506100
Name: w_label, dtype: int64

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# **Random Forest**

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
forest = RandomForestClassifier(n_estimators = 50, min_samples_leaf = 5)

In [15]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train, y_train, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([23.34409928, 22.31900215, 18.66093659, 23.73352289, 22.39670324,
        21.71089578, 22.17227077, 20.55814505, 23.51160693, 20.49991012]),
 'score_time': array([0.27951956, 0.27452087, 0.26972604, 0.28100562, 0.27879167,
        0.27899146, 0.27687836, 0.277596  , 0.28085256, 0.27528596]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

# **Random Forest(SMOTE-ENN)**

In [16]:
from imblearn.combine import SMOTEENN

SMOTEENN = SMOTEENN(random_state=0)
x_train_over,y_train_over = SMOTEENN.fit_resample(x_train,y_train)

print('SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트: ', x_train_over.shape, y_train_over.shape)
print('SMOTEENN 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트:  (955920, 5) (955920,)
SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트:  (1304846, 5) (1304846,)
SMOTEENN 적용 후 레이블 값 분포: 
 0.0    652423
1.0    652423
Name: w_label, dtype: int64


In [17]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train_over, y_train_over, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([36.57737184, 31.51783776, 37.88840413, 33.94403601, 33.95321751,
        28.38056707, 31.85881495, 35.6059103 , 33.48289704, 33.42577815]),
 'score_time': array([0.32394671, 0.30583692, 0.32166219, 0.30478239, 0.31355119,
        0.29774737, 0.31042957, 0.31715846, 0.30478096, 0.31136298]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}